# Experiment: Train/Tune BERT Model

## Confirm Environment

In [ ]:
!conda info

## Setup and Imports

In [ ]:
from emolex.preprocessing import load_mental_health_sentiment_dataset, clean_text, encode_sentiment_labels, split_data, dl_text_vectorization
from emolex.dl_models import bilstm_model 
from emolex.train import train_dl_model
from emolex.evaluation import plot_training_history, generate_confusion_matrix, generate_classification_report
from emolex.utils import detect_and_set_device

## Device Setup

In [ ]:
# Detect and set up GPU or use CPU
device_used = detect_and_set_device()
print(f"TensorFlow is configured to use: {device_used}")

## Load Data

In [ ]:
df = load_mental_health_sentiment_dataset()
df.info()
df.head()

## Clean Data

In [ ]:
print(f"\n--- Cleaning Text ---")
df['clean_text'] = df["text"].apply(clean_text)
print("Text cleaning complete. Sample cleaned text:")
print("\n", df[["text", "clean_text"]].sample(5))

## Encode Labels

In [ ]:
print(f"\n--- Encoding Labels ---")
df, encoder = encode_sentiment_labels(df)
print("Label encoding complete. Sample encoded labels:")
print("\n", df[['label', 'label_encoded']].sample(5))

## Train-Test Split

In [ ]:
print("\n--- Perform Train-Test Split ---")
X_train_raw, X_test_raw, y_train, y_test = split_data(df) 
print(f"Train set size: {len(X_train_raw)} samples")
print(f"Test set size: {len(X_test_raw)} samples")

## Vectorization

In [ ]:
print("\n--- Performing Text Vectoriation ---")
X_train_pad_filtered, X_test_pad_filtered, y_train_filtered, y_test_filtered = dl_text_vectorization(X_train_raw, X_test_raw, y_train, y_test)
print("Vecorization complete.")

## Build Model

## Train Model

## Evaluate Model

In [ ]:
print("\n--- Plot Training History ---")
plot_training_history(history)

In [ ]:
print("\n--- Predict Test Classes ---")
y_pred = model.predict(X_test_pad_filtered)
y_pred_classes = y_pred.argmax(axis=1)

In [ ]:
print("\n--- Generate Confusion Matrix ---")
fig, ax = generate_confusion_matrix(y_test_filtered, y_pred_classes, class_labels=encoder.classes_)

In [ ]:
print("\n--- Generate Classification Report ---")
generate_classification_report(y_test_filtered, y_pred_classes, class_labels=encoder.classes_)